# Assignment 2 - Rajas Thakur

In [2]:
import numpy as np
from sklearn import decomposition
from scipy import linalg
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
import spacy
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

In [132]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [133]:
cd 'drive/My Drive/AppliedNLP/HW2'

[Errno 2] No such file or directory: 'drive/My Drive/AppliedNLP/HW2'
/content/drive/My Drive/AppliedNLP/HW2


### Exploratory Data Analysis and Data Preprocessing 

In [134]:
train1 = pd.read_csv('train.csv')

In [135]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89970 entries, 0 to 89969
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Id                     89970 non-null  int64  
 1   country                89927 non-null  object 
 2   description            89970 non-null  object 
 3   designation            64086 non-null  object 
 4   points                 89970 non-null  int64  
 5   price                  83828 non-null  float64
 6   province               89927 non-null  object 
 7   region_1               75108 non-null  object 
 8   region_2               34869 non-null  object 
 9   taster_name            71843 non-null  object 
 10  taster_twitter_handle  68373 non-null  object 
 11  title                  89970 non-null  object 
 12  variety                89970 non-null  object 
 13  winery                 89970 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 9.6+

In [136]:
# Plotting a correlation matrix
corr=train1.corr()
corr.style.background_gradient(cmap='coolwarm')

,Id,points,price
Id,1.000000,0.006112,0.002072
points,0.006112,1.000000,0.407324
price,0.002072,0.407324,1.000000


In [137]:
# Checking for null values in train dataset
train1.isna().any()

Id                       False
country                   True
description              False
designation               True
points                   False
price                     True
province                  True
region_1                  True
region_2                  True
taster_name               True
taster_twitter_handle     True
title                    False
variety                  False
winery                   False
dtype: bool

In [138]:
# Checking the percentage of missing data in train data
train1.isna().sum()/train1.shape[0]

Id                       0.000000
country                  0.000478
description              0.000000
designation              0.287696
points                   0.000000
price                    0.068267
province                 0.000478
region_1                 0.165188
region_2                 0.612437
taster_name              0.201478
taster_twitter_handle    0.240047
title                    0.000000
variety                  0.000000
winery                   0.000000
dtype: float64

In [139]:
public = pd.read_csv('public.csv')
private = pd.read_csv('private.csv')

In [140]:
public.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20001 entries, 0 to 20000
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Id                     20001 non-null  int64  
 1   country                19992 non-null  object 
 2   description            20001 non-null  object 
 3   designation            14201 non-null  object 
 4   points                 20001 non-null  int64  
 5   price                  18523 non-null  float64
 6   province               19992 non-null  object 
 7   region_1               16870 non-null  object 
 8   region_2               7830 non-null   object 
 9   taster_name            15924 non-null  object 
 10  taster_twitter_handle  15196 non-null  object 
 11  title                  20001 non-null  object 
 12  variety                20000 non-null  object 
 13  winery                 20001 non-null  object 
 14  Unnamed: 14            1 non-null      float64
dtypes:

In [141]:
private.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Id                     20000 non-null  int64  
 1   country                19989 non-null  object 
 2   description            20000 non-null  object 
 3   designation            14219 non-null  object 
 4   points                 20000 non-null  int64  
 5   price                  18624 non-null  float64
 6   province               19989 non-null  object 
 7   region_1               16746 non-null  object 
 8   region_2               7812 non-null   object 
 9   taster_name            15960 non-null  object 
 10  taster_twitter_handle  15189 non-null  object 
 11  title                  20000 non-null  object 
 12  variety                20000 non-null  object 
 13  winery                 20000 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 2.1+

In [142]:
# Checking the percentage of missing data in train data
public.isna().sum()/public.shape[0]

Id                       0.000000
country                  0.000450
description              0.000000
designation              0.289986
points                   0.000000
price                    0.073896
province                 0.000450
region_1                 0.156542
region_2                 0.608520
taster_name              0.203840
taster_twitter_handle    0.240238
title                    0.000000
variety                  0.000050
winery                   0.000000
Unnamed: 14              0.999950
dtype: float64

In [143]:
# Checking the percentage of missing data in train data
private.isna().sum()/private.shape[0]

Id                       0.00000
country                  0.00055
description              0.00000
designation              0.28905
points                   0.00000
price                    0.06880
province                 0.00055
region_1                 0.16270
region_2                 0.60940
taster_name              0.20200
taster_twitter_handle    0.24055
title                    0.00000
variety                  0.00000
winery                   0.00000
dtype: float64

In [144]:
# We observe that there is big percentage of null values in these datasets for columns: Designation, Region_1, Region_2, Taster_name, Taster_twitter_handle
# Hence we will drop these columns from our datasets.
train1.drop(['designation','region_1','region_2','taster_name','taster_twitter_handle'],axis=1,inplace= True)
public.drop(['designation','region_1','region_2','taster_name','taster_twitter_handle','Unnamed: 14'],axis=1,inplace= True)
private.drop(['designation','region_1','region_2','taster_name','taster_twitter_handle'],axis=1,inplace= True)

In [145]:
# We still observe that there are few missing values in the columns: country, price and province. 
# We will use dropna to remove such rows from our data. If we impute price with Mean or Median, it will introduce bias in the model.
train1.dropna(subset=['country','price','province'],inplace=True)

In [146]:
public.dropna(subset=['country','price','province','variety'],inplace=True)
private.dropna(subset=['country','price','province'],inplace=True)

In [147]:
train1.head()

,Id,country,description,points,price,province,title,variety,winery
0,69712,France,"A buoyant wine, this has delicious acidity and...",89,17.0,Provence,Domaine de la Sanglière 2015 La Riviera Rosé (...,Rosé,Domaine de la Sanglière
1,1171,Germany,"Despite weighing in at 12% alcohol, this seems...",86,20.0,Rheinhessen,Louis Guntrum 2006 Niersteiner Bergkirche Kabi...,Riesling,Louis Guntrum
2,129181,US,"Sweet, simple and fruity, this light-bodied wi...",84,23.0,California,Naggiar 2016 Estate Muscat Canelli (Sierra Foo...,Muscat Canelli,Naggiar
3,116814,US,Tastes like a sweet and sour sauce mixed into ...,84,19.0,California,One Hope 2009 Sauvignon Blanc (California),Sauvignon Blanc,One Hope
4,116519,Spain,As a one-liter package you get one-third more ...,86,9.0,Central Spain,Bodegas Eguren 2007 Tempranillo (Vino de la Ti...,Tempranillo,Bodegas Eguren


In [148]:
# Checking for unique values of Country column
train1['country'].unique()

array(['France', 'Germany', 'US', 'Spain', 'Italy', 'Chile',
       'New Zealand', 'Moldova', 'Argentina', 'Portugal', 'Israel',
       'Australia', 'Austria', 'Canada', 'Hungary', 'South Africa',
       'Peru', 'Greece', 'Turkey', 'Romania', 'Mexico', 'Slovenia',
       'Macedonia', 'Cyprus', 'Georgia', 'England', 'Lebanon', 'Uruguay',
       'Bulgaria', 'Morocco', 'Ukraine', 'Croatia', 'Switzerland',
       'Brazil', 'Serbia', 'Luxembourg', 'Slovakia', 'Czech Republic',
       'India', 'Bosnia and Herzegovina', 'Armenia', 'China'],
      dtype=object)

In [149]:
# Checking for unique values of Province column
train1['province'].unique()

array(['Provence', 'Rheinhessen', 'California', 'Central Spain',
       'Sicily & Sardinia', 'Maule Valley', 'Southern Italy',
       'Washington', 'Northeastern Italy', 'Mosel', 'Oregon',
       'Marlborough', 'Bordeaux', 'Iowa', 'Moldova', 'Rheingau',
       'Mendoza Province', 'Dão', 'Northern Spain', 'Awatere Valley',
       'Douro', 'Alsace', 'Limarí Valley', 'Galilee', 'Idaho', 'Veneto',
       'New York', 'South Australia', 'Burgenland', 'Burgundy',
       'Languedoc-Roussillon', 'Piedmont', 'Tuscany', 'Rhône Valley',
       'Champagne', 'Colchagua Valley', 'France Other', 'Catalonia',
       'Bairrada', 'British Columbia', 'Maipo Valley', 'Tejo', 'Other',
       'Loire Valley', 'Kamptal', 'Villány', 'Judean Hills',
       'Central Italy', 'Vinho Verde', 'Beaujolais', 'Codru Region',
       'Kremstal', 'Virginia', 'Casablanca Valley', 'Niederösterreich',
       'Southwest France', 'New Mexico', 'Eisenberg', 'Australia Other',
       'Alentejano', 'Walker Bay', 'Tokaji', 'Stellen

In [150]:
# Checking for unique values of Variety column
train1['variety'].unique()

array(['Rosé', 'Riesling', 'Muscat Canelli', 'Sauvignon Blanc',
       'Tempranillo', 'Pinot Noir', 'Carricante', 'Carmenère', 'Merlot',
       'Zinfandel', 'Greco', 'Pinot Bianco', 'Roussanne', 'Chardonnay',
       'White Blend', 'Cabernet Sauvignon', 'Petit Verdot',
       'Pinot Grigio', 'Sangiovese', 'Bonarda', 'Portuguese Red',
       'Tinta de Toro', 'Red Blend', 'Muscat', 'Chenin Blanc', 'Syrah',
       'Gewürztraminer', 'Cabernet Franc', 'Cabernet Blend',
       'Bordeaux-style Red Blend', 'Sparkling Blend', 'Mencía',
       'Charbono', 'Sauvignon Blanc-Chardonnay', 'Grenache',
       'Rhône-style Red Blend', 'Barbera', 'Rhône-style White Blend',
       'Lagrein', 'Champagne Blend', 'Viognier', 'Malbec', 'Primitivo',
       'Jacquère', 'Bical', 'Pinot Blanc', 'Dolcetto',
       'Viognier-Chardonnay', 'Mourvèdre', 'Torrontés', 'Melon',
       'Nuragus', 'Grüner Veltliner', 'Encruzado', 'Aligoté',
       'Vermentino', 'Nebbiolo', 'Prosecco', 'Alvarinho', 'Shiraz',
       'Viura',

In [151]:
# Checking for unique values of Title column
train1['title'].unique()

array(['Domaine de la Sanglière 2015 La Riviera Rosé (Côtes de Provence)',
       'Louis Guntrum 2006 Niersteiner Bergkirche Kabinett Riesling (Rheinhessen)',
       'Naggiar 2016 Estate Muscat Canelli (Sierra Foothills)', ...,
       'Quinta do Vesuvio 2010 Pombal do Vesúvio Red (Douro)',
       'Domaine Roger Champault 2014 Les Pierris  (Sancerre)',
       'Noble Vines 2015 667 Pinot Noir (Monterey)'], dtype=object)

In [152]:
# Checking for unique values of Winery column
train1['winery'].unique()

array(['Domaine de la Sanglière', 'Louis Guntrum', 'Naggiar', ...,
       'Château Bonnin', 'Dixie Flyer Vineyards', 'Lucien Crochet'],
      dtype=object)

In [153]:
# As we see above, there are many unique values for columns: Province, Variety and Winery. 
# If we try to convert these into categorical variables, the model will become complicated with many variables.
# Hence, we will drop these columns from our dataset
train1.drop(['province','Id','variety','winery','title'],axis=1,inplace= True)
public.drop(['province','Id','variety','winery','title'],axis=1,inplace= True)
private.drop(['province','Id','variety','winery','title'],axis=1,inplace= True)

In [154]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83789 entries, 0 to 89969
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   country      83789 non-null  object 
 1   description  83789 non-null  object 
 2   points       83789 non-null  int64  
 3   price        83789 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.2+ MB


In [155]:
# Checking for unique values of Country column
train1['country'].unique()

array(['France', 'Germany', 'US', 'Spain', 'Italy', 'Chile',
       'New Zealand', 'Moldova', 'Argentina', 'Portugal', 'Israel',
       'Australia', 'Austria', 'Canada', 'Hungary', 'South Africa',
       'Peru', 'Greece', 'Turkey', 'Romania', 'Mexico', 'Slovenia',
       'Macedonia', 'Cyprus', 'Georgia', 'England', 'Lebanon', 'Uruguay',
       'Bulgaria', 'Morocco', 'Ukraine', 'Croatia', 'Switzerland',
       'Brazil', 'Serbia', 'Luxembourg', 'Slovakia', 'Czech Republic',
       'India', 'Bosnia and Herzegovina', 'Armenia', 'China'],
      dtype=object)

In [156]:
public['country'].unique()

array(['Portugal', 'Argentina', 'US', 'Italy', 'Chile', 'Germany',
       'Spain', 'France', 'Austria', 'New Zealand', 'South Africa',
       'Morocco', 'Greece', 'Australia', 'Canada', 'Uruguay', 'Bulgaria',
       'Croatia', 'Israel', 'Hungary', 'Turkey', 'Luxembourg', 'Mexico',
       'Serbia', 'Moldova', 'Georgia', 'England', 'Romania', 'India',
       'Slovenia', 'Lebanon', 'Brazil', 'Ukraine', 'Czech Republic',
       'Switzerland', 'Cyprus'], dtype=object)

In [157]:
private['country'].unique()

array(['US', 'Argentina', 'France', 'New Zealand', 'Chile', 'Italy',
       'Spain', 'Romania', 'Australia', 'Germany', 'Georgia', 'Portugal',
       'Austria', 'Greece', 'South Africa', 'Israel', 'Canada', 'Hungary',
       'Brazil', 'England', 'Bulgaria', 'Mexico', 'Uruguay', 'Croatia',
       'Lebanon', 'Switzerland', 'Ukraine', 'Moldova', 'Turkey',
       'Macedonia', 'Morocco', 'Peru', 'Slovenia', 'Cyprus', 'Armenia',
       'India', 'Serbia'], dtype=object)

In [158]:
# Since we observe that there multiple countries, we will divide them into continents.
Europe = ['France','Germany','Spain', 'Italy','Moldova', 'Portugal', 
       'Austria', 'Hungary','Greece', 'Turkey', 'Romania', 'Slovenia',
       'Macedonia', 'Cyprus', 'Georgia', 'England',
       'Bulgaria', 'Ukraine', 'Croatia', 'Switzerland',
       'Serbia', 'Luxembourg', 'Slovakia', 'Czech Republic',
       'Bosnia and Herzegovina', 'Armenia']

Asia = ['India','China','Israel','Japan','Lebanon','South Korea']

Australia = ['Australia','New Zealand']

NorthAmerica = ['Canada','US','Mexico']

SouthAmerica = ['Uruguay','Chile','Brazil','Argentina']

Africa = ['South Africa','Morocco']

In [159]:
train1['continent'] = np.nan
public['continent'] = np.nan
private['continent'] = np.nan

In [160]:
# We will create a function for replacing country with continent
def replace_country(df):

  for x in range(len(df['country'])):
    if df['country'].iloc[x] in Europe:
      df['continent'].iloc[x] = 'Europe'
    elif df['country'].iloc[x] in Asia:
      df['continent'].iloc[x] = 'Asia'
    elif df['country'].iloc[x] in Australia:
      df['continent'].iloc[x] = 'Australia'
    elif df['country'].iloc[x] in NorthAmerica:
      df['continent'].iloc[x] = 'NorthAmerica'
    elif df['country'].iloc[x] in SouthAmerica:
      df['continent'].iloc[x] = 'SouthAmerica'
    elif df['country'].iloc[x] in Africa:
      df['continent'].iloc[x] = 'Africa'

In [161]:
replace_country(train1)
replace_country(private)
replace_country(public)

In [162]:
train1.head()

,country,description,points,price,continent
0,France,"A buoyant wine, this has delicious acidity and...",89,17.0,Europe
1,Germany,"Despite weighing in at 12% alcohol, this seems...",86,20.0,Europe
2,US,"Sweet, simple and fruity, this light-bodied wi...",84,23.0,NorthAmerica
3,US,Tastes like a sweet and sour sauce mixed into ...,84,19.0,NorthAmerica
4,Spain,As a one-liter package you get one-third more ...,86,9.0,Europe


In [163]:
public.head()

,country,description,points,price,continent
0,Portugal,A lemon flavored wine that's crisp and fruity....,85,17.0,Europe
1,Argentina,Waves of berry aromas matched by oak-based van...,90,31.0,SouthAmerica
2,US,). Very good wine from a winery increasingly k...,92,47.0,NorthAmerica
3,Italy,"Light gold in color, this opens with easy arom...",85,9.0,Europe
4,US,"This wine has jazzy, jammy aromas and ripe bla...",91,40.0,NorthAmerica


In [164]:
private.head()

,country,description,points,price,continent
0,US,"The nose is broad and welcoming, with apple an...",87,30.0,NorthAmerica
2,US,"Floral and herbal from the get-go, this medium...",91,23.0,NorthAmerica
4,Argentina,"Quite spicy, with the Syrah element offering a...",91,38.0,SouthAmerica
5,US,Not to be confused with the winery's Vintner's...,89,39.0,NorthAmerica
6,France,"From a producer in the Layon valley, this is a...",84,15.0,Europe


In [165]:
continent=pd.get_dummies(train1['continent'],columns='continent',prefix='continent')
train1=pd.concat([train1, continent], axis=1)
train1.drop(['continent'],axis=1,inplace= True)

In [166]:
continent1=pd.get_dummies(public['continent'],columns='continent',prefix='continent')
public=pd.concat([public, continent1], axis=1)
public.drop(['continent'],axis=1,inplace= True)

In [167]:
continent2=pd.get_dummies(private['continent'],columns='continent',prefix='continent')
private=pd.concat([private, continent2], axis=1)
private.drop(['continent'],axis=1,inplace= True)

## Task 1 
### 1. Running Linear Regression Model with L2 (Ridge) Penalty

In [ ]:
#Subsampling data
#train2 = train1
train2 = train1.sample(n=15000,random_state=0)

In [ ]:
X = train1.drop(["description","points","country"],axis=1)
y = train1['points']

In [ ]:
X

,price,continent_Africa,continent_Asia,continent_Australia,continent_Europe,continent_NorthAmerica,continent_SouthAmerica
0,17.0,0,0,0,1,0,0
1,20.0,0,0,0,1,0,0
2,23.0,0,0,0,0,1,0
3,19.0,0,0,0,0,1,0
4,9.0,0,0,0,1,0,0
...,...,...,...,...,...,...,...
89965,15.0,0,0,0,1,0,0
89966,11.0,0,0,0,0,0,1
89967,28.0,0,0,0,1,0,0
89968,23.0,0,0,0,1,0,0


In [ ]:
y

0        89
1        86
2        84
3        84
4        86
         ..
89965    90
89966    84
89967    89
89968    88
89969    86
Name: points, Length: 83789, dtype: int64

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train_org, X_test_org, y_train, y_test = train_test_split(X, y, random_state = 0)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_org)
X_test = scaler.transform(X_test_org)

In [ ]:
from sklearn.model_selection import GridSearchCV
from  sklearn.linear_model import Ridge

param_grid = {'alpha':[0.01, 0,0.1, 1, 10]}

In [ ]:
Grid_Ridge = GridSearchCV(Ridge(),param_grid,cv=10,return_train_score=True)
Grid_Ridge.fit(X_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [0.01, 0, 0.1, 1, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring=None, verbose=0)

In [ ]:
# Finding out the best parameters for the model

print("Best Cross-Validation Score: {:.4f}".format(Grid_Ridge.best_score_))
print('Best Paramater:',Grid_Ridge.best_params_)

Best Cross-Validation Score: 0.1721
Best Paramater: {'alpha': 0.1}


In [ ]:
# Reporting the training and testing scores for the best model

print("Best Training score: {:.4f}".format(Grid_Ridge.score(X_train,y_train)))
print("Best Testing score: {:.4f}".format(Grid_Ridge.score(X_test,y_test)))

Best Training score: 0.1796
Best Testing score: 0.1941


### 2. Creating Text based model

In [ ]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83789 entries, 0 to 89969
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   country                 83789 non-null  object 
 1   description             83789 non-null  object 
 2   points                  83789 non-null  int64  
 3   price                   83789 non-null  float64
 4   continent_Africa        83789 non-null  uint8  
 5   continent_Asia          83789 non-null  uint8  
 6   continent_Australia     83789 non-null  uint8  
 7   continent_Europe        83789 non-null  uint8  
 8   continent_NorthAmerica  83789 non-null  uint8  
 9   continent_SouthAmerica  83789 non-null  uint8  
dtypes: float64(1), int64(1), object(2), uint8(6)
memory usage: 3.7+ MB


In [ ]:
X1 = train1.sample(n=10000,random_state=0)

In [ ]:
X1

,country,description,points,price,continent_Africa,continent_Asia,continent_Australia,continent_Europe,continent_NorthAmerica,continent_SouthAmerica
41700,US,Soft and still fairly tannic at the age of fou...,87,35.0,0,0,0,0,1,0
12763,New Zealand,"A dark, savory style of Pinot Noir, with inten...",87,23.0,0,0,1,0,0,0
23454,Italy,Rich almost to the point of chunky; aromas of ...,93,70.0,0,0,0,1,0,0
83900,Germany,Smoky and savory with a touch of spring ramps ...,89,21.0,0,0,0,1,0,0
13184,US,This 100% varietal wine from this new winery o...,91,55.0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
16784,Italy,"This opens with aromas of rose petal, lychee a...",89,38.0,0,0,0,1,0,0
31086,US,"The aromas of huckleberry, flowers and blueber...",89,40.0,0,0,0,0,1,0
5077,Italy,"Aromas of French oak, coconut, exotic spice an...",89,50.0,0,0,0,1,0,0
44433,US,"As fabulous as ever, this brilliant compendium...",95,35.0,0,0,0,0,1,0


In [ ]:
y1 = X1['points']

In [ ]:
y1

41700    87
12763    87
23454    93
83900    89
13184    91
         ..
16784    89
31086    89
5077     89
44433    95
20389    92
Name: points, Length: 10000, dtype: int64

In [ ]:
X1 = X1['description']

In [ ]:
X1

41700    Soft and still fairly tannic at the age of fou...
12763    A dark, savory style of Pinot Noir, with inten...
23454    Rich almost to the point of chunky; aromas of ...
83900    Smoky and savory with a touch of spring ramps ...
13184    This 100% varietal wine from this new winery o...
                               ...                        
16784    This opens with aromas of rose petal, lychee a...
31086    The aromas of huckleberry, flowers and blueber...
5077     Aromas of French oak, coconut, exotic spice an...
44433    As fabulous as ever, this brilliant compendium...
20389    A crisp bouquet of orange zest marks the openi...
Name: description, Length: 10000, dtype: object

In [ ]:
# Converting to lower case
X1=X1.str.lower()

In [ ]:
# Replacing period with space
X1 = X1.str.replace("\."," ")

In [ ]:
# Removing extra spaces
X1 = X1.apply(lambda x:re.sub('\s+',' ',x))

In [ ]:
# Lemmatization
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def lemmatization_function_spacy(text):
  tokens = nlp(text)
  lemm_tokens = [x.lemma_ for x in tokens]
  return ' '.join(lemm_tokens)

In [ ]:
X1 = X1.apply(lemmatization_function_spacy)

In [ ]:
X1

41700    soft and still fairly tannic at the age of fou...
12763    a dark , savory style of pinot noir , with int...
23454    rich almost to the point of chunky ; aroma of ...
83900    smoky and savory with a touch of spring ramps ...
13184    this 100 % varietal wine from this new winery ...
                               ...                        
16784    this open with aroma of rose petal , lychee an...
31086    the aroma of huckleberry , flower and blueberr...
5077     aromas of french oak , coconut , exotic spice ...
44433    as fabulous as ever , this brilliant compendiu...
20389    a crisp bouquet of orange z mark the opening o...
Name: description, Length: 10000, dtype: object

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X1, y1, random_state=0)

In [ ]:
# Running regression model in pipeline
pipe = Pipeline([('count',CountVectorizer()),
                     ('LR',Ridge(max_iter=10000))])

param_grid = {'LR__alpha': [0.01,0, 0.1, 1, 10]}

In [ ]:
LR_CV = GridSearchCV(pipe, param_grid, cv=10,n_jobs=-1)

In [ ]:
LR_CV.fit(X_train, y_train)
print("Best cross-validation score: {:.4f}".format(LR_CV.best_score_))

Best cross-validation score: 0.6153


In [ ]:
LR_CV.best_params_
print("Train score: {:.2f}".format(LR_CV.score(X_train, y_train)))
print("Test score: {:.4f}".format(LR_CV.score(X_test, y_test)))

Train score: 0.81
Test score: 0.6236


### 3. Trying n-grams and using TFIDF

In [ ]:
# Running regression model in pipeline with n-grams
pipe = Pipeline([('count',CountVectorizer()),
                     ('LR',Ridge(max_iter=10000))])

param_grid = {'LR__alpha': [0.01,0, 0.1, 1, 10],
              'count__ngram_range':[(1,1),(1,2),(2,2)]}

In [ ]:
LR_CV = GridSearchCV(pipe, param_grid, cv=5,n_jobs=-1)

In [ ]:
LR_CV.fit(X_train, y_train)
print("Best cross-validation score: {:.4f}".format(LR_CV.best_score_))

Best cross-validation score: 0.6117


In [ ]:
LR_CV.best_params_
print("Train score: {:.2f}".format(LR_CV.score(X_train, y_train)))
print("Test score: {:.4f}".format(LR_CV.score(X_test, y_test)))

Train score: 0.81
Test score: 0.6236


In [ ]:
# We see that using n-grams hasn't improved the accuracy. Trying TFIDF vectorizer
# Running regression model in pipeline with n-grams
pipe = Pipeline([('tfidf',TfidfVectorizer()),
                     ('LR',Ridge(max_iter=10000))])

param_grid = {'LR__alpha': [0, 0.01, 0.1, 1, 10],
              'tfidf__ngram_range':[(1,1),(1,2),(2,2)],
              'tfidf__max_df':[0.1,0.5,0.75,0.95],
              'tfidf__min_df':[1,2,5]}

In [ ]:
LR_TV = GridSearchCV(pipe, param_grid, cv=5,n_jobs=-1)

In [ ]:
LR_TV.fit(X_train, y_train)
print("Best cross-validation score: {:.4f}".format(LR_TV.best_score_))

Best cross-validation score: 0.5944


In [ ]:
LR_TV.best_params_
print("Train score: {:.2f}".format(LR_TV.score(X_train, y_train)))
print("Test score: {:.4f}".format(LR_TV.score(X_test, y_test)))

Train score: 0.83
Test score: 0.6152


### 4. Combining the two models

In [ ]:
X1

41700    soft and still fairly tannic at the age of fou...
12763    a dark , savory style of pinot noir , with int...
23454    rich almost to the point of chunky ; aroma of ...
83900    smoky and savory with a touch of spring ramps ...
13184    this 100 % varietal wine from this new winery ...
                               ...                        
16784    this open with aroma of rose petal , lychee an...
31086    the aroma of huckleberry , flower and blueberr...
5077     aromas of french oak , coconut , exotic spice ...
44433    as fabulous as ever , this brilliant compendiu...
20389    a crisp bouquet of orange z mark the opening o...
Name: description, Length: 10000, dtype: object

In [ ]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83789 entries, 0 to 89969
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   country                 83789 non-null  object 
 1   description             83789 non-null  object 
 2   points                  83789 non-null  int64  
 3   price                   83789 non-null  float64
 4   continent_Africa        83789 non-null  uint8  
 5   continent_Asia          83789 non-null  uint8  
 6   continent_Australia     83789 non-null  uint8  
 7   continent_Europe        83789 non-null  uint8  
 8   continent_NorthAmerica  83789 non-null  uint8  
 9   continent_SouthAmerica  83789 non-null  uint8  
dtypes: float64(1), int64(1), object(2), uint8(6)
memory usage: 3.7+ MB


In [ ]:
X2 = train1.sample(n=10000,random_state=0)

In [ ]:
y2 = X2['points']

In [ ]:
X2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 41700 to 20389
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   price                   10000 non-null  float64
 1   continent_Africa        10000 non-null  uint8  
 2   continent_Asia          10000 non-null  uint8  
 3   continent_Australia     10000 non-null  uint8  
 4   continent_Europe        10000 non-null  uint8  
 5   continent_NorthAmerica  10000 non-null  uint8  
 6   continent_SouthAmerica  10000 non-null  uint8  
dtypes: float64(1), uint8(6)
memory usage: 214.8 KB


In [ ]:
y2

41700    87
12763    87
23454    93
83900    89
13184    91
         ..
16784    89
31086    89
5077     89
44433    95
20389    92
Name: points, Length: 10000, dtype: int64

In [ ]:
X2 = X2.drop(["description","points","country"],axis=1)

In [ ]:
X2

,price,continent_Africa,continent_Asia,continent_Australia,continent_Europe,continent_NorthAmerica,continent_SouthAmerica
41700,35.0,0,0,0,0,1,0
12763,23.0,0,0,1,0,0,0
23454,70.0,0,0,0,1,0,0
83900,21.0,0,0,0,1,0,0
13184,55.0,0,0,0,0,1,0
...,...,...,...,...,...,...,...
16784,38.0,0,0,0,1,0,0
31086,40.0,0,0,0,0,1,0
5077,50.0,0,0,0,1,0,0
44433,35.0,0,0,0,0,1,0


In [ ]:
y2

41700    87
12763    87
23454    93
83900    89
13184    91
         ..
16784    89
31086    89
5077     89
44433    95
20389    92
Name: points, Length: 10000, dtype: int64

In [ ]:
X1

41700    soft and still fairly tannic at the age of fou...
12763    a dark , savory style of pinot noir , with int...
23454    rich almost to the point of chunky ; aroma of ...
83900    smoky and savory with a touch of spring ramps ...
13184    this 100 % varietal wine from this new winery ...
                               ...                        
16784    this open with aroma of rose petal , lychee an...
31086    the aroma of huckleberry , flower and blueberr...
5077     aromas of french oak , coconut , exotic spice ...
44433    as fabulous as ever , this brilliant compendiu...
20389    a crisp bouquet of orange z mark the opening o...
Name: description, Length: 10000, dtype: object

In [ ]:
CV = CountVectorizer()
X_matrix = CV.fit_transform(X1)

In [ ]:
X_matrix

<10000x9882 sparse matrix of type '<class 'numpy.int64'>'
	with 343880 stored elements in Compressed Sparse Row format>

In [ ]:
from scipy.sparse import hstack
X_data = hstack([X_matrix,X2])

In [ ]:
X_data

<10000x9889 sparse matrix of type '<class 'numpy.float64'>'
	with 363877 stored elements in COOrdinate format>

In [ ]:
# Splitting data into train and test
X_train, X_test, y_train, y_test = train_test_split(X_data, y2, random_state = 0)

In [ ]:
# Running regression model
params ={'alpha':[0.01, 0.1, 0, 1,10]}
model = GridSearchCV(Ridge(), params, cv=5, return_train_score = True)
model.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [0.01, 0.1, 0, 1, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring=None, verbose=0)

In [ ]:
print("Best Cross-validation score: {:.2f}".format(model.best_score_))
print('Best parameters: ', model.best_params_)
print("Train Score: {:.4f} ".format(model.score(X_train,y_train)))
print("Test Score: {:.4f} ".format(model.score(X_test,y_test)))

Best Cross-validation score: 0.64
Best parameters:  {'alpha': 10}
Train Score: 0.8284 
Test Score: 0.6613 


## Task 2 - Pre Trained Word Vectors

In [4]:
from gensim.models import Word2Vec, KeyedVectors

In [3]:
train1 = pd.read_csv('train.csv')

In [5]:
train2 = train1.sample(n=10000,random_state=0,replace=True)

In [7]:
# Getting description column
X3 = train2['description']

In [8]:
# We will perform data cleaning in the steps below
# Converting to lower case
X3=X3.str.lower()

In [9]:
# Replacing period with space
X3 = X3.str.replace("\."," ")

In [10]:
# Removing extra spaces
X3 = X3.apply(lambda x:re.sub('\s+',' ',x))

In [13]:
# Lemmatization
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rajas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
def lemmatization_function(text):
  tokens = text.split()
  lemm_tokens = [lemmatizer.lemmatize(x) for x in tokens]
  return ' '.join(lemm_tokens)

In [15]:
X3 = X3.apply(lemmatization_function)

In [23]:
y3 = train2['points']

In [16]:
# Using word tokenizer to convert sentences into word tokens
sentences=[]
sentences = [nltk.word_tokenize(sentence) for sentence in X3]

In [18]:
# Removing stop words
from nltk.corpus import stopwords
nltk.download('stopwords')

for i in range(len(sentences)):
    sentences[i] = [word for word in sentences[i] if word not in stopwords.words('english')]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rajas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
Word2VecModel = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin',binary=True, limit =100000)

In [25]:
# Creating a function for calculating avergae value of word vector for a sentence
def AvgSentenceWeight(sentence):
  sentence = [word for word in sentence if word in Word2VecModel.wv.vocab]
  return np.mean(Word2VecModel[sentence],axis=0)

In [26]:
X3 = X3.apply(AvgSentenceWeight)

c:\python\python37\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
X3 = list(X3)

In [28]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = train_test_split(X3, y3, random_state = 0)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [29]:
from sklearn.model_selection import GridSearchCV
from  sklearn.linear_model import Ridge

param_grid = {'alpha':[0.01, 0,0.1, 1, 10]}

In [30]:
Grid_Ridge = GridSearchCV(Ridge(),param_grid,cv=10,return_train_score=True)
Grid_Ridge.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=Ridge(),
             param_grid={'alpha': [0.01, 0, 0.1, 1, 10]},
             return_train_score=True)

In [31]:
# Finding out the best parameters for the model

print("Best Cross-Validation Score: {:.4f}".format(Grid_Ridge.best_score_))
print('Best Paramater:',Grid_Ridge.best_params_)

Best Cross-Validation Score: 0.1890
Best Paramater: {'alpha': 10}


In [32]:
# Reporting the training and testing scores for the best model

print("Best Training score: {:.4f}".format(Grid_Ridge.score(X_train,y_train)))
print("Best Testing score: {:.4f}".format(Grid_Ridge.score(X_test,y_test)))

Best Training score: 0.1975
Best Testing score: 0.1807


## Task 3 - Custom Embeddings

In [168]:
# Checking original cleaned dataset
train1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83789 entries, 0 to 89969
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   country                 83789 non-null  object 
 1   description             83789 non-null  object 
 2   points                  83789 non-null  int64  
 3   price                   83789 non-null  float64
 4   continent_Africa        83789 non-null  uint8  
 5   continent_Asia          83789 non-null  uint8  
 6   continent_Australia     83789 non-null  uint8  
 7   continent_Europe        83789 non-null  uint8  
 8   continent_NorthAmerica  83789 non-null  uint8  
 9   continent_SouthAmerica  83789 non-null  uint8  
dtypes: float64(1), int64(1), object(2), uint8(6)
memory usage: 3.7+ MB


In [169]:
# Sampling 15000 observations
train2 = train1.sample(n=15000,random_state=0)

In [170]:
train2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 41700 to 1694
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   country                 15000 non-null  object 
 1   description             15000 non-null  object 
 2   points                  15000 non-null  int64  
 3   price                   15000 non-null  float64
 4   continent_Africa        15000 non-null  uint8  
 5   continent_Asia          15000 non-null  uint8  
 6   continent_Australia     15000 non-null  uint8  
 7   continent_Europe        15000 non-null  uint8  
 8   continent_NorthAmerica  15000 non-null  uint8  
 9   continent_SouthAmerica  15000 non-null  uint8  
dtypes: float64(1), int64(1), object(2), uint8(6)
memory usage: 673.8+ KB


In [171]:
import nltk
import re
from gensim.models import Word2Vec
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [172]:
train2['description'].head()

41700    Soft and still fairly tannic at the age of fou...
12763    A dark, savory style of Pinot Noir, with inten...
23454    Rich almost to the point of chunky; aromas of ...
83900    Smoky and savory with a touch of spring ramps ...
13184    This 100% varietal wine from this new winery o...
Name: description, dtype: object

In [173]:
# Getting description column
X3 = train2['description']

In [174]:
# We will perform data cleaning in the steps below
# Converting to lower case
X3=X3.str.lower()

In [175]:
# Replacing period with space
X3 = X3.str.replace("\."," ")

In [176]:
# Removing extra spaces
X3 = X3.apply(lambda x:re.sub('\s+',' ',x))

In [177]:
# Lemmatization
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [178]:
def lemmatization_function_spacy(text):
  tokens = nlp(text)
  lemm_tokens = [x.lemma_ for x in tokens]
  return ' '.join(lemm_tokens)

In [179]:
X3 = X3.apply(lemmatization_function_spacy)

In [180]:
# Using word tokenizer to convert sentences into word tokens
sentences=[]
sentences = [nltk.word_tokenize(sentence) for sentence in X3]

In [181]:
# Removing stop words
for i in range(len(sentences)):
    sentences[i] = [word for word in sentences[i] if word not in stopwords.words('english')]

In [182]:
# Creating a customize embeddings using CBOW method(by default) for Word2Vec 
model = Word2Vec(sentences, min_count=2)

In [183]:
# Initializing vocabulary in words 
words = model.wv.vocab

In [184]:
# Checking length of our vocabulary
len(words)

6937

In [185]:
# Finding Word Vectors for word 'wine'
vector = model.wv['wine']
print(vector)

[-0.33385035 -0.8475174  -0.15607741  0.08587971 -0.503473    0.1892408
  0.6345584   0.08645012  0.7339715  -0.2252895  -0.13428481  0.15229368
  0.56223434 -0.64435494 -0.5334693   0.32160512 -0.21347573  0.11488811
  0.3877721  -0.7677564   0.59119064  0.73388505 -0.9868946  -0.7366786
  1.3183991  -0.16988337 -0.4858496  -0.43563253 -0.10064092 -0.22582911
  0.44409272  1.3371855  -1.1644049  -0.7461012   0.3719654   0.78156626
  0.03987685  0.62235254 -0.42470288 -0.6246745   0.7094606  -0.01464618
  0.31484643  0.42213476 -0.61608034 -0.11533866  0.00931252  0.5309613
 -0.0356039   0.10980927 -0.28233513 -0.63865703 -0.31922922  0.50392896
  0.7006034  -0.74400383  1.1633749   0.0031386  -0.19326274 -0.24369249
  0.0068918  -0.82341784  0.8367924  -0.53334457  0.12146839  0.17428577
  0.1788078   0.4870763   0.52062184  0.27239668 -0.8604824  -0.82413286
  0.28563076 -0.655094   -0.3828846  -0.029964   -0.43908495  0.7222465
  0.2579452   0.03881463 -0.4144165   0.4551676   0.803

In [186]:
# Creating a function for calculating avergae value of word vector for a sentence
def AvgSentenceWeight(sentence):
  sentence = [word for word in sentence if word in words]
  return np.mean(model[sentence],axis=0)

In [187]:
X3

41700    soft and still fairly tannic at the age of fou...
12763    a dark , savory style of pinot noir , with int...
23454    rich almost to the point of chunky ; aroma of ...
83900    smoky and savory with a touch of spring ramps ...
13184    this 100 % varietal wine from this new winery ...
                               ...                        
51939    hail from 11 different vineyard locate across ...
80640    entice aroma of woodland berry , crush rise , ...
65398    vintner gaetano bertani deliver a dark and del...
83510    lean and tight than this bottling have be the ...
1694     the northernmost beaujolais cru of saint - amo...
Name: description, Length: 15000, dtype: object

In [188]:
# Applying the function for our data 
Xdata = X3.apply(AvgSentenceWeight)

In [189]:
Xdata

41700    [0.081530534, 0.1558283, -0.024640031, 0.11930...
12763    [0.11498597, 0.17200464, -0.0049973414, 0.0161...
23454    [0.12823018, 0.19261324, 0.00018380086, 0.0231...
83900    [0.11856767, 0.19200495, -0.012854189, 0.06151...
13184    [0.14910603, 0.20815657, -0.032135386, 0.10732...
                               ...                        
51939    [0.13307132, 0.18980841, -0.04201915, 0.091334...
80640    [0.11583395, 0.1888407, -0.013542376, 0.065468...
65398    [0.09680354, 0.15136309, -0.0144967185, 0.0374...
83510    [0.107724436, 0.17955634, -0.014147795, 0.0928...
1694     [0.16662113, 0.24261633, 0.027173333, 0.095824...
Name: description, Length: 15000, dtype: object

In [190]:
# Converting our data to a list
Xdata = list(Xdata)

In [191]:
Xdata

[array([ 0.08153053,  0.1558283 , -0.02464003,  0.11930456, -0.12631941,
         0.06813134, -0.03509311,  0.02763368, -0.04862771, -0.09860413,
        -0.11808739,  0.08233406, -0.04318614,  0.01921346, -0.02486231,
        -0.05457878,  0.08991459, -0.08731481,  0.01786513,  0.09784958,
         0.07883516,  0.06979577,  0.09850612, -0.02432243, -0.06060379,
        -0.03708918, -0.0177942 ,  0.07400269, -0.07686346, -0.04892782,
        -0.00904443,  0.01699213, -0.02994   , -0.00770669, -0.03034406,
        -0.04139231, -0.19055468,  0.00144253, -0.05354655, -0.08033137,
         0.02477206,  0.10956345, -0.10029104,  0.06487267,  0.0247147 ,
        -0.17129567,  0.10006413, -0.06440414,  0.12423201, -0.02478342,
         0.06517166,  0.01632237,  0.00524993,  0.14318012, -0.01249637,
        -0.05140198, -0.10753275, -0.13708006, -0.09775291, -0.0124685 ,
        -0.0004671 , -0.12786438, -0.00227416,  0.00923749,  0.03030122,
         0.06069959, -0.14148384,  0.06076008,  0.0

In [192]:
ydata = train2['points']

In [193]:
ydata

41700    87
12763    87
23454    93
83900    89
13184    91
         ..
51939    89
80640    91
65398    90
83510    90
1694     89
Name: points, Length: 15000, dtype: int64

In [194]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = train_test_split(Xdata, ydata, random_state = 0)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [195]:
from sklearn.model_selection import GridSearchCV
from  sklearn.linear_model import Ridge

param_grid = {'alpha':[0.01, 0,0.1, 1, 10]}

In [196]:
Grid_Ridge = GridSearchCV(Ridge(),param_grid,cv=10,return_train_score=True)
Grid_Ridge.fit(X_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [0.01, 0, 0.1, 1, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring=None, verbose=0)

In [197]:
# Finding out the best parameters for the model

print("Best Cross-Validation Score: {:.4f}".format(Grid_Ridge.best_score_))
print('Best Paramater:',Grid_Ridge.best_params_)

Best Cross-Validation Score: 0.3711
Best Paramater: {'alpha': 0}


In [198]:
# Reporting the training and testing scores for the best model

print("Best Training score: {:.4f}".format(Grid_Ridge.score(X_train,y_train)))
print("Best Testing score: {:.4f}".format(Grid_Ridge.score(X_test,y_test)))

Best Training score: 0.3831
Best Testing score: 0.3693


### We observe that this model with custom embeddings performs better than pre-trained word embeddings.